THIS IS THE NOTEBOOK THAT I USED FOR FINETUNING THE MODEL. PLEASE USE THIS NOTEBOOK FOR FOLLWING ALONG IN THE LECTURE.

In [ ]:
!nvidia-smi

Sat Aug 26 18:49:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#### THIS COMMAND IS REQUIRED ONLY IF YOU WANT TO STORE YOUR TRAINED MODEL TO YOUR GOOGLE DRIVE.
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git


Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607
Receiving objects: 100% (607/607), 27.84 MiB | 18.81 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [ ]:
!ls

alpaca-lora  gdrive  sample_data


In [ ]:
%cd alpaca-lora

/content/alpaca-lora


In [ ]:
!ls

alpaca_data_cleaned_archive.json  generate.py
alpaca_data_gpt4.json		  lengths.ipynb
alpaca_data.json		  LICENSE
DATA_LICENSE			  pyproject.toml
docker-compose.yml		  README.md
Dockerfile			  requirements.txt
export_hf_checkpoint.py		  templates
export_state_dict_checkpoint.py   utils
finetune.py


In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_json("alpaca_data.json")

In [ ]:
df

,instruction,input,output
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...
...,...,...,...
51997,Generate an example of what a resume should li...,,"Jean Tremaine\n1234 Main Street, Anytown, CA 9..."
51998,Arrange the items given below in the order to ...,"cake, me, eating",I eating cake.
51999,Write an introductory paragraph about a famous...,Michelle Obama,Michelle Obama is an inspirational woman who r...
52000,Generate a list of five things one should keep...,,1. Research potential opportunities and carefu...


In [ ]:
len(df)

52002

In [ ]:
df_small = df[0:5000]

In [ ]:
df_small.to_json('temp.json', orient='records')

In [ ]:
!ls

alpaca_data_cleaned_archive.json  generate.py
alpaca_data_gpt4.json		  lengths.ipynb
alpaca_data.json		  LICENSE
DATA_LICENSE			  pyproject.toml
docker-compose.yml		  README.md
Dockerfile			  requirements.txt
export_hf_checkpoint.py		  temp.json
export_state_dict_checkpoint.py   templates
finetune.py			  utils


In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-smpt2z97
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-smpt2z97
  Resolved https://github.com/huggingface/peft.git to commit 8c17d556a8fe9522e10d73d7bd3fad46a6ecae14
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python finetune.py \
    --base_model 'openlm-research/open_llama_3b_v2' \
    --data_path './temp.json' \
    --output_dir './lora-alpaca-temp' \
    --batch_size 16 \
    --micro_batch_size 16 \
    --num_epochs 2 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

2023-08-26 18:50:35.987985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Training Alpaca-LoRA model with params:
base_model: openlm-research/open_llama_3b_v2
data_path: ./temp.json
output_dir: ./lora-alpaca-temp
batch_size: 16
micro_batch_size: 16
num_epochs: 2
learning_rate: 0.0001
cutoff_len: 512
val_set_size: 2000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: alpaca

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you under

In [ ]:
!ls

alpaca_data_cleaned_archive.json  lengths.ipynb
alpaca_data_gpt4.json		  LICENSE
alpaca_data.json		  lora-alpaca-temp
DATA_LICENSE			  pyproject.toml
docker-compose.yml		  README.md
Dockerfile			  requirements.txt
export_hf_checkpoint.py		  temp.json
export_state_dict_checkpoint.py   templates
finetune.py			  utils
generate.py


In [ ]:
%pwd

'/content/alpaca-lora'

In [ ]:
%cd /content/alpaca-lora

/content/alpaca-lora


In [ ]:
import os

path = "/content/gdrive/MyDrive/mysafefolder" ## THIS IS THE PATH WHERE THE FINETUNED MODEL WILL BE STORED IN YOUR GOOGLE DRIVE
os.mkdir(path)

In [ ]:
!cp -r ../alpaca-lora/ /content/gdrive/MyDrive/mysafefolder ## COPYING THE MODEL WEIGHTS TO THE PATH

In [ ]:
%cd /content/gdrive/MyDrive/mysafefolder/alpaca-lora

***BELOW IS THE COMMAND FOR GENERATING THE UI FOR DOING INFERENCE FROM OUR FINETUNED MODEL***

---



In [ ]:
!python generate.py \
    --base_model 'openlm-research/open_llama_3b_v2' \
    --lora_weights 'lora-alpaca-temp' \
    --share_gradio True